In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [30]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

In [3]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
data = pd.read_csv('/kaggle/input/megathon-context-qna/train_context.csv')
qna = pd.read_csv('/kaggle/input/megathon-context-qna/train_qna.csv')

In [6]:
df = pd.merge(data, qna, left_on='index', right_on='context', how='inner')
df.head()

,index,context_x,context_y,question,answer
0,0,"Gastritis is an inflammation, irritation, or ...",0,What are the symptoms of gastritis?,"However, the most common symptoms include: Nau..."
1,0,"Gastritis is an inflammation, irritation, or ...",0,What does the treatment for gastritis involve?,Treatment for gastritis usually involves: Taki...
2,0,"Gastritis is an inflammation, irritation, or ...",0,How does an upper endoscopy help diagnose gast...,"However, the most common symptoms include: Nau..."
3,0,"Gastritis is an inflammation, irritation, or ...",0,How is a fecal occult blood test (stool test) ...,This test checks for the presence of blood in ...
4,0,"Gastritis is an inflammation, irritation, or ...",0,What is gastritis?,"Gastritis is an inflammation, irritation, or ..."


In [7]:
context_question = []
for index, row in df.iterrows():
    context_question.append('Question: ' + row['question'] + '\n\nAnswer:' +  '\n\nContext: ' + row['context_x'])

In [8]:
len(context_question)

27728

In [41]:
inputs = tokenizer(context_question, 
                   return_tensors="pt", 
                   padding='max_length', 
                   truncation=True,
                    max_length=2500,
                  ).to('cuda')

Answer = df['answer'][:100].tolist()

outputs = tokenizer(Answer, 
                   return_tensors="pt", 
                   padding='max_length', 
                   truncation=True,
                    max_length=2500,
                  ).to('cuda')


train_dataset = TensorDataset(torch.tensor(inputs.input_ids), torch.tensor(outputs.input_ids))

/tmp/ipykernel_32/55086056.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(inputs.input_ids), torch.tensor(outputs.input_ids))


In [10]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, inputs):
        self.inputs = inputs

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        return {key: self.inputs[key][idx] for key in self.inputs}

In [11]:
dataset = CustomDataset(inputs)
batch_size = 1  # Adjust the batch size as needed
dataloader = DataLoader(dataset, batch_size=batch_size)

In [13]:
torch.cuda.empty_cache() 

In [ ]:
for batch in dataloader:
    batch = {key: batch[key].to('cuda') for key in batch}
    with torch.no_grad():
        outputs = model.generate(**batch).to('cuda')
#     answers.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))
        torch.cuda.empty_cache()  # Clear GPU cache

    print(tokenizer.batch_decode(outputs, skip_special_tokens=True, max_length=200))
# outputs = model.generate(**inputs).to('cuda')
# print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

In [15]:
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

In [16]:
!pip install git+https://github.com/huggingface/peft.git

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-_9b3kwc3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-_9b3kwc3
  Resolved https://github.com/huggingface/peft.git to commit 207229ad5e1854cd897306ce5ee725b0245b1064
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.6.0.dev0-py3-none-any.whl size=124489 sha256=af694246c6df624c0bce4fc160f3a01cb74e155846256a26cba705a0a136c6f0
  Stored in directory: /tmp/pip-ephem-wheel-cache-xj6n2d93/wheels/d7/c7/de/1368fac8

In [17]:
from peft import LoraConfig, get_peft_model

In [21]:
config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q",
        "k",
        "v",
        "o",
        "wi_0",
        "wi_1",
        "wo",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

In [23]:
model=get_peft_model(model, config)

In [24]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): T5ForConditionalGeneration(
          (shared): Embedding(32128, 1024)
          (encoder): T5Stack(
            (embed_tokens): Embedding(32128, 1024)
            (block): ModuleList(
              (0): T5Block(
                (layer): ModuleList(
                  (0): T5LayerSelfAttention(
                    (SelfAttention): T5Attention(
                      (q): Linear(
                        in_features=1024, out_features=1024, bias=False
                        (lora_dropout): ModuleDict(
                          (default): Dropout(p=0.05, inplace=False)
                        )
                        (lora_A): ModuleDict(
                          (default): Linear(in_features=1024, out_features=32, bias=False)
                        )
                        (lora_B): ModuleDict(
                          (default): Linear(in_features=32, o

In [26]:
if torch.cuda.device_count() > 1:
    model.is_parallelizable = True
    model.model_parallel = True

In [58]:
import transformers
from datetime import datetime

project = "journal-finetune"
base_model_name = "flan_T5"
run_name = base_model_name + '-' + project
output_dir = "./"+run_name

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset = train_dataset,
    #data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    args = transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=1,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        max_steps=500,
        learning_rate=2.5e-5,
        optim="adafactor",
        logging_steps=25,
        evaluation_strategy="steps",
        save_steps=25
    )
)

In [50]:
!pip install bnb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Using cached bnb-0.3.0-py3-none-any.whl (9.7 kB)
  Using cached attrs-20.3.0-py2.py3-none-any.whl (49 kB)
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Using cached PyYAML-5.4.1.tar.gz (175 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [62 lines of output]
      /tmp/pip-build-env-7mfmsbxc/overlay/lib/python3.10/site-packages/setuptools/config/setupcfg.py:293: _DeprecatedConfig: Deprecated config in `setup.cfg`
      !!
      
              ****************************************

In [59]:
model.config.use_cache = False
trainer.train()

TypeError: vars() argument must have __dict__ attribute